# Assignment 6 - Graph drawing and network analysis

In this assignment you will get to work with graphs using the [graph-tool python module](https://graph-tool.skewed.de/).

Learning goals are:

* perform basic network analysis to create derived graph properties,
* visualize graphs and their properties.

If you require an introduction to basic graph notions, consider to read the so-titled section in [Maël Fabien
's introductory blog post](https://towardsdatascience.com/introduction-to-graphs-part-1-2de6cda8c5a5). While graph theory and algorithms can lead to fairly deep mathematical discussion, basic definitions and much practical use of graph data are quite easy to get started with.

For further background on `graph-tool` refer to the [graph tutorial slides](https://stevenbergner.github.io/Teaching/cmpt767/graph-tutorial) and the [tutorial notebook](https://stevenbergner.github.io/Teaching/cmpt767/graph-tool-tutorial-todo.ipynb). See the [lab setup](#Environment-setup) section below to learn about the `gt conda` environment that has all required dependencies installed in the Big Data Lab.

In [1]:
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown
%matplotlib inline

import graph_tool.all as gt
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

graph-tool version: 2.29


# Problem 1: Power grid analysis

In [2]:
g = gt.collection.data['power']
display(Markdown(gt.collection.descriptions['power']))

Power grid: An undirected, unweighted network representing the topology of the Western States Power Grid of the United States. Data compiled by D. Watts and S. Strogatz and made available on the web `here <http://cdg.columbia.edu/cdg/datasets>`_. Please cite D. J. Watts and S. H. Strogatz, Nature 393, 440-442 (1998). Retrieved from `Mark Newman's website <http://www-personal.umich.edu/~mejn/netdata/>`_.

 In this graph an edge represents a power supply line. A node is either a generator, a transformator, or a substation.
 
**Task 1a**: Create a drawing of this graph that emphasizes nodes that have more than 10 incident power supply lines. Set the size of all other nodes to 0, but retain visibility of the power lines.

In [3]:
# TODO
# # this assigns random values to the vertex properties
# vprop_double.get_array()[:] = random(g.num_vertices())

# # or more conveniently (this is equivalent to the above)
# vprop_double.a = random(g.num_vertices())

deg= g.degree_property_map("total")

for i in range(len(deg.a)):
    if deg.a[i] <= 10:
        deg.a[i] = 0 


pos = gt.sfdp_layout(g)
gt.graph_draw(g, pos=pos, vertex_size=deg,  edge_pen_width=1.2,output="graph-draw.png")


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f522c6dd358, at 0x7f522c73a240>

**Task 1b:** Identify one of the centrality measures that can be used to indicate powerlines that act as a bridge between different parts of the network. Use this to emphasize structurally important nodes and powerlines.

In [4]:
# TODO
vp, ep = gt.betweenness(g)
gt.graph_draw(g, pos=g.vp["pos"], vertex_fill_color=vp,
               vertex_size=gt.prop_to_size(vp, mi=1, ma=15),
               edge_pen_width=gt.prop_to_size(ep, mi=0.5, ma=5),
               vcmap=plt.cm.gist_heat,
               vorder=vp, output="polblogs_betweenness.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f522c6dd358, at 0x7f522c6dd3c8>

# Problem 2: Small social graph visualization

In [5]:
X_knows = {
    'Mary': ['Peter', 'Albert', 'DavidF', 'Peter'],
    'Judy': ['Bob', 'Alan'],
    'Peter': ['Mary', 'DavidF', 'Jon'],
    'DavidF': ['Albert', 'Joseph', 'Peter', 'Mary'],
    'Jon': ['Peter', 'Joseph', 'DavidE'],
    'DavidE': ['Jon', 'Joseph', 'Albert'],
    'Joseph': ['DavidE', 'Jon', 'DavidF'],
    'Bob': ['Judy', 'Alan'],
    'Alan': ['Bob', 'Mary', 'Judy'],
    'Albert': ['DavidF', 'Mary', 'DavidE'],
}

**Task**: Create an undirected graph based on the information above, remove parallel edges, and draw it using a layout that resembles the tidy example given below.

In [6]:
# TODO
from graph_tool.all import *
names = [key for key in X_knows.keys()]

edges_list = [sorted([i, j]) for i in X_knows for j in X_knows[i]]

social_g = gt.Graph(directed=False)

edges_list = set(tuple(x) for x in edges_list)
edges_list = [ list(x) for x in edges_list ]


new_names_list = []
for i in range(len(edges_list)):
    new_names_list = new_names_list + edges_list[i]      
final_list = []
for i in range(len(new_names_list)):
    if new_names_list[i] not in final_list:
        final_list.append(new_names_list[i])
print(final_list)

social_g.add_edge_list(edges_list,hashed=True, string_vals=True)

# print(social_g.vertex)
# print(social_g.vertex(0))
v_prop = social_g.new_vertex_property("string")
for i in range(social_g.num_vertices()):
    v_prop[social_g.vertex(i)] = final_list[i]

social_g.vertex_properties['social_name']= v_prop
graph_draw(social_g, vertex_text=social_g.vertex_properties['social_name'], vertex_font_size=10,                                                                                                         
            output_size=(300, 300), output="social_graph.png")


['DavidE', 'Joseph', 'Jon', 'DavidF', 'Peter', 'Alan', 'Mary', 'Albert', 'Bob', 'Judy']


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f51f5ef4f60, at 0x7f51f5ed22b0>

# Problem 3: Facebook graph analysis

For the following question let us work with a real social graph of facebook friendship connections. Please download [`facebook_combined.txt` from SNAP](https://snap.stanford.edu/data/ego-Facebook.html), the Stanford Large Network Dataset Collection and create a `Graph` object with `graph-tool`. The dataset contains the ego networks of 10 facebook users, i.e. the friends of each of these users and the connections among those friends.

Goal of the following questions is to use centrality measures to determine influencers among the users, not including the ego nodes themselves.

**Task 3a:** Load the dataset and create a drawing of the graph.

In [8]:
# TODO
data = pd.read_csv('facebook_combined.txt', sep=" ", header=None)
data.columns = ['first','second']
# data

,first,second
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
5,0,6
6,0,7
7,0,8
8,0,9
9,0,10


In [9]:
# TODO
# Create an empty list 
total_list =[] 
  
# Iterate over each row 
for index, rows in data.iterrows():
    # Create list for the current row 
    single_list =[rows['first'], rows['second']] 
      
    # append the list to the final list 
    total_list.append(sorted(single_list)) 
  
total_list = [set(x) for x in total_list]
total_list = [ list(x) for x in total_list ]

fb_g = gt.Graph(directed=False)
fb_g.add_edge_list(total_list,hashed=True, string_vals=False)

print(fb_g.vertex)
graph_draw(fb_g, vertex_font_size=1,output_size=(1300, 1300), output="fb_graph.png")


<bound method Graph.vertex of <Graph object, undirected, with 4039 vertices and 88234 edges at 0x7f51f5ea16a0>>


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f51f5ea16a0, at 0x7f51f5ed2e10>

**Task 3b:** Calculate and show a histogram of pairwise distances among users, i.e. on the shortest paths of friendship connections among any pair of users. Use a log-scale to show the frequencies in the histogram. What is the diameter of this graph?

*Hint:* Calculating and drawing the histogram amounts to two lines of code using `gt` and `plt` modules.

In [ ]:
# TODO

**Task 3c:** Determine influencers, i.e. people withing the ego network that are well connected among friends.

There are several steps to this analysis. First, remove the ego nodes. You can do this in an approximate way, simply by calculating a measure of influence of a user and removing the highest scoring nodes, assuming that these naturally are the ego nodes whose friends this network consists of.

Use `PageRank` as a measure of influence of a node.

*Step 1* - Create a [`GraphView`](https://graph-tool.skewed.de/static/doc/quickstart.html#graph-views) and drawing that only retains nodes with a `pagerank` $< 0.002$.

In [27]:
# TODO
pr = gt.pagerank(fb_g)
# print(type(pr.a))
print(pr)
v_prop = fb_g.new_vertex_property("double")
print(fb_g.num_vertices())
for i in range(fb_g.num_vertices()):
    v_prop[fb_g.vertex(i)] = pr.a[i]
    
fb_gv = gt.GraphView(fb_g, vfilt = lambda v: v_prop[v] < 0.002)


# filtered_pr = []
# for i in range(len(pr.a)):
#     if pr.a[i] < 0.002:
#         filtered_pr.append(pr.a[i])
# filtered_pr = np.asarray(filtered_pr)


pr2 = gt.pagerank(fb_gv)

pos = sfdp_layout(fb_gv)

gt.graph_draw(fb_gv, pos=pos,vertex_fill_color=pr2,
              vertex_size=gt.prop_to_size(pr2, mi=1, ma=15),
              vcmap=plt.cm.gist_heat,
               output="fb_pr.png")

<VertexPropertyMap object with value type 'double', for Graph 0x7f51f5ea16a0, at 0x7f51f50cce80>
4039


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f51f5e98668, at 0x7f51f50cc668>

In [ ]:
# TODO

*Step 2* - Only retain the largest connected component of this graph, i.e. create another [`GraphView`](https://graph-tool.skewed.de/static/doc/quickstart.html#graph-views) of only this largest component.

In [16]:
# TODO
fb_gv2 = gt.GraphView(g, vfilt=gt.label_largest_component(g))
pr = gt.pagerank(fb_gv2)
print(pr)
print(type(pr))

gt.graph_draw(fb_gv2, pos=g.vp["pos"], vertex_fill_color=pr,
               vertex_size=gt.prop_to_size(pr, mi=1, ma=15),
               vorder=pr, vcmap=plt.cm.gist_heat,
               output="fb_large_component.png")

<VertexPropertyMap object with value type 'double', for Graph 0x7f51f5e9d978, at 0x7f522c6dd4a8>
<class 'graph_tool.VertexPropertyMap'>


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f522c6dd358, at 0x7f522c6dd3c8>

*Step 3* - Recalculate pagerank for each node, choose a threshold (e.g. 0.0005) above which only a small part of the users are selected. Create a drawing that emphasizes these users that potentially play central roles among their friends.

In [ ]:
# TODO

In [ ]:
# TODO

# Environment setup

## Lab setup
We have prepared a `conda` environment in the lab that contains all modules required for this assignment. You can activate the environment as follows:

```
conda activate /usr/shared/CMPT/big-data/condaenv/gt
```
You should now be able to `import graph_tool` in a python 3 session or inside jupyter notebooks.

#### You might get an error
`CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.`
This happens, if you have not used conda before. To perform initialization once, we recommend the following steps:
```
conda init bash
conda config --set auto_activate_base false
```
If you now restart your terminal shell or simply invoke a new one by typing `bash`, you can try activating the environment, again.

#### Reproduce the conda setup in the lab

In case you want to reproduce the setup, here are the calls used to create the environment:
```
conda create -n gt
conda activate gt
conda install -c conda-forge graph-tool
conda install -c pkgw-forge -c conda-forge gtk3 pygobject
conda install -c conda-forge jupyter jupyterlab
conda install -c conda-forge matplotlib scikit-learn seaborn networkx igraph pyintergraph graphviz
```


## Setup on Linux (Ubuntu or WSL)

If you have sudo privileges, you can simply run
```
sudo apt install python3-graph-tool
```
and proceed to install further Python dependencies via `pip`, not requiring anaconda.
In case there are issues with drawing functionality, you may also need
```
sudo install libcairo2-dev xdg-utils graphviz
```

Note, that this also works on Windows' WSL (Windows Sub-shell for Linux) running Ubuntu, in case you do not want to launch a VM for this.

## Other environments and remote use

Setting up `graph_tool` on Mac OSX is possible via `brew`, but evidence online suggests that this may not be straightforward.
Since the [conda-forge graph-tool package](https://anaconda.org/conda-forge/graph-tool) is available for `osx-64`, maybe you want to try installing via `conda` instead of `brew`.

Remote use of the lab setup is possible, but tricky. Familiarize yourself with a shell session manager, such as [`tmux`](https://tmuxcheatsheet.com/) or GNU [`screen`](https://gist.github.com/fredrick/1216878) to keep an ssh session open once you logged into a lab machine. To connect to a jupyter notebook remotely you could create a [`ngrok`](https://ngrok.com/) account and use their http port tunneling. If you have questions about this method, ask Steven in the lab for more hints.


# Submission instructions
Please submit [this notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A6/A6.ipynb) with filled out code cells and figures included to [CourSys activity A6](https://coursys.sfu.ca/2020sp-cmpt-733-g1/+a6/).